In [58]:
# Imports
from buildingmotif import BuildingMOTIF
from rdflib import Namespace, Literal
from buildingmotif.dataclasses import Model
from buildingmotif.dataclasses import Library
from buildingmotif.namespaces import BRICK, REF
import csv
import json
import os

In [59]:
# Binding namespaces and initializing model
VLTT = Namespace('https://www.doesntexist.org/volttronsemantics#')
BLDG = Namespace('urn:bldg/')
bm = BuildingMOTIF("sqlite://") # in-memory instance
model = Model.create(BLDG, description="Test Model") 
model.graph.bind("vltt", VLTT)
model.graph.bind("ref", REF)

In [60]:
# getting libraries and templates
brick = Library.load(ontology_graph="../BuildingMOTIF/libraries/brick/Brick-full.ttl")
lib = Library.load(directory="vltt-templates")
topic_name = lib.get_template_by_name("volttron-topic")
point_list = lib.get_template_by_name("volttron-point-list")
device_config = lib.get_template_by_name("volttron-device-config")
bacnet_lib = Library.load(directory='../BuildingMOTIF/libraries/bacnet')
bacnet_point_template = bacnet_lib.get_template_by_name("brick-point")

2023-03-12 23:18:43,432 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice
2023-03-12 23:18:43,433 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject


In [61]:
bacnet_device_template = bacnet_lib.get_template_by_name('brick-device')

In [71]:
#read csv files and create list of dicts
def read_csv(path):
    line = 0
    contents = []
    with open(path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            contents.append(row)
        return contents

# get templates using list of dictionaries from read_csv
def get_templates(dct_list, key):
    templates = {}
    type_set = {item[key] for item in dct_list}
    for type in type_set:
        templates[type] = brick.get_template_by_name(BRICK[type])
    return templates

# add points given an item dictionary from the entity table
def add_points(item, filepath, model):
    points = read_csv(filepath)
    templates = get_templates(points, 'Brick Type')
    name = item.get('URI')

    for point in points:
        type = point.get('Brick Type')
        point_name = name + '.' + point.get('Point Name')
        topic= item.get('base_topic') + '/' + point.get('Point Name')
        # print(point_name)
        model.add_graph(
            templates[type].evaluate(
                {
                    "name":BLDG[point_name]
                }
            )
        )
        model.add_graph(
            topic_name.evaluate(
                {
                    "name":BLDG[point_name + '.' + 'topic'],
                    "topic":Literal(topic)
                }
            )
        )
        model.graph.add((BLDG[item['URI']], BRICK.hasPoint, BLDG[point_name]))
        model.graph.add((BLDG[point_name], REF['hasExternalReference'], BLDG[point_name + '.topic']))

def add_brick_relations(filepath, model):
    triples = read_csv(filepath)
    for triple in triples:
        model.graph.add((BLDG[triple['subjectURI']], BRICK[triple['relation']], BLDG[triple['objectURI']]))


# For examples/IoT

In [72]:
entity_table_path  = 'examples/IoT_site/entity_table.csv'
relation_table_path = 'examples/IoT_site/entity_relations.csv'
point_list_dir = 'examples/IoT_site/volttron_configs'
contents = read_csv(entity_table_path)
templates = get_templates(contents, 'type')

In [76]:
#Instantiate templates and add to model
for item in contents:
    type = item.get('type')
    name = item.get('URI')
    # print(name)

    model.add_graph(
        templates[type].evaluate(
            {
                "name":BLDG[name]
            }
        )
    )

    if item.get('point_list', None):
        model.add_graph(
            point_list.evaluate(
                {
                    "name":BLDG[name+".point_list"],
                    "config-path": Literal(item.get('point_list'))
                }
            )
        )
        model.graph.add((BLDG[name], REF['hasExternalReference'], BLDG[name+".point_list"]))

        add_points(item, os.path.join(point_list_dir,item.get('point_list')), model)

    if item.get('config', None):
        model.add_graph(
            device_config.evaluate(
                {
                    "name":BLDG[name+".config"],
                    "config-path": Literal(item.get('config'))
                }
            )
        )
        model.graph.add((BLDG[name], REF['hasExternalReference'], BLDG[name+".config"]))

In [77]:
add_brick_relations(relation_table_path, model)

In [78]:
print(model.graph.serialize('IoT_model.ttl'))

<84e174f6-3aa0-4b02-9085-9e44ea5aa339> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'SQLAlchemy'].


# For BACnet (BOPtest) example

## Creating model using volttron topics

In [38]:
model_bacnet.graph.bind('bacnet', Namespace('http://data.ashrae.org/bacnet/2020#'))
model_bacnet.graph.bind('brick', BRICK)

rdflib.namespace.Namespace

In [39]:
model_bacnet = Model.create(BLDG, description="Test BACnet Model") 
model_bacnet.graph.bind("vltt", VLTT)
model_bacnet.graph.bind("ref", REF)
model_bacnet.graph.bind('bacnet', Namespace('http://data.ashrae.org/bacnet/2020#'))
model_bacnet.graph.bind('brick', BRICK)

In [40]:
entity_table_path  = 'examples/BACnet/entity_table.csv'
relation_table_path = 'examples/BACnet/'
point_list_dir = 'examples/BACnet/'
contents = read_csv(entity_table_path)
templates = get_templates(contents, 'type')

In [41]:
#Instantiate templates and add to model_bacnet
for item in contents:
    type = item.get('type')
    name = item.get('URI')
    # print(name)

    model_bacnet.add_graph(
        templates[type].evaluate(
            {
                "name":BLDG[name]
            }
        )
    )

    if item.get('point_list', None):
        model_bacnet.add_graph(
            point_list.evaluate(
                {
                    "name":BLDG[name+".point_list"],
                    "config-path": Literal(item.get('point_list'))
                }
            )
        )
        model_bacnet.graph.add((BLDG[name], REF['hasExternalReference'], BLDG[name+".point_list"]))

        add_points(item, os.path.join(point_list_dir,item.get('point_list')), model_bacnet)

    if item.get('config', None):
        model_bacnet.add_graph(
            device_config.evaluate(
                {
                    "name":BLDG[name+".config"],
                    "config-path": Literal(item.get('config'))
                }
            )
        )
        model_bacnet.graph.add((BLDG[name], REF['hasExternalReference'], BLDG[name+".config"]))

In [42]:
model_bacnet.graph.print()

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix vltt: <https://www.doesntexist.org/volttronsemantics#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/building> a brick:Building .

<urn:bldg/hvacZone> a brick:Zone ;
    brick:hasPoint <urn:bldg/hvacZone.ahu_reaFloSupAir_y_bacnet>,
        <urn:bldg/hvacZone.ahu_reaPFanRet_y_bacnet>,
        <urn:bldg/hvacZone.ahu_reaPFanSup_y_bacnet>,
        <urn:bldg/hvacZone.ahu_reaTRetAir_y_bacnet>,
        <urn:bldg/hvacZone.oveTZonSet_u_bacnet>,
        <urn:bldg/hvacZone.reaCO2Zon_y_bacnet>,
        <urn:bldg/hvacZone.reaPEle_y_bacnet>,
        <urn:bldg/hvacZone.reaPPum_y_bacnet>,
        <urn:bldg/hvacZone.reaQHea_y_bacnet>,
        <urn:bldg/hvacZone.reaTZon_y_bacnet> ;
    ref:hasExternalReference <urn:bldg/hvacZone.config>,
        <urn:bldg/hvacZone.point_list> .

<urn:bldg/hvacZone.ahu_reaFloSupAir_y_bacnet.topic> a vltt:

## adding information using ref schema BACnetReference

In [43]:
# add BACnet information to graph
def create_BACnet_device(config_file, device_name, model):
    with open(config_file,'r') as f:
        config = json.load(f)
    inputs = {
        'address': Literal(config.get('driver_config').get('device_address')),
        'instance-number': Literal(config.get('driver_config').get('device_id')),
        'name': BLDG[device_name + '.bacnet.device']
    }
    model.add_graph(
        bacnet_device_template.evaluate(inputs)
    )
    
def add_BACnet_refs(item, filepath, device_name, model):
    points = read_csv(filepath)
    name = item.get('URI')

    for point in points:
        bacnet_point_name = name + '.' + point.get('Point Name')
        inputs = {
            'name': BLDG[bacnet_point_name],
            'identifier': Literal(point.get('BACnetObjectType')),
            'obj-name': Literal(point.get('BACnetPointName')),
            'units': Literal(point.get('Units')),
            'device': BLDG[device_name + '.bacnet.device']
        }
            # print(point_name)
        model.add_graph(
            bacnet_point_template.evaluate(inputs)
            )
        
        model.graph.add((BLDG[point.get('Point Name')], REF['hasExternalReference'], BLDG[bacnet_point_name]))


In [44]:
bacnet_file_path = 'examples/BACnet/bacnet.config'
for item in contents:
    if item.get('config') == bacnet_file_path:
        device_name = item.get('URI')
        create_BACnet_device(bacnet_file_path, device_name, model_bacnet)
        print(os.path.join(point_list_dir,item.get('point_list')))
        add_BACnet_refs(item, os.path.join(point_list_dir,item.get('point_list')), device_name, model_bacnet)


examples/BACnet/point_list.csv


In [47]:
model_bacnet.graph.serialize('bacnet_model.ttl')

<Graph identifier=f627445a-a45e-4038-933d-740d891c64dc (<class 'rdflib.graph.Graph'>)>